In [4]:
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
import scvi

In [5]:
# --- Path Configuration ---
ref_path = r"E:\UIC_PHD\Python_projects\parquet_read\src\map_h5ad\refs\bladder_ref.h5ad"
query_path = r"F:\12_30_2025_test_azimuth_annotation\HBM262.LMHQ.472.h5ad"

In [6]:
adata = sc.read_h5ad(query_path)
ref_data = sc.read_h5ad(ref_path)

In [7]:
adata.var_names = adata.var_names.str.split('.').str[0]
ref_data.var_names = ref_data.var_names.str.split('.').str[0]

In [8]:
common_genes = adata.var_names.intersection(ref_data.var_names)
adata_sub = adata[:, common_genes].copy()
ref_sub = ref_data[:, common_genes].copy()

In [9]:
adata_sub = adata_sub[:, ref_sub.var_names].copy()

In [10]:
sc.pp.pca(ref_sub)
sc.pp.neighbors(ref_sub, n_neighbors=14, use_rep='X_pca')
sc.tl.umap(ref_sub, random_state = 42)

In [11]:
sc.tl.ingest(adata_sub, ref_sub, obs='cell_type')

E:\UIC_PHD\Python_projects\parquet_read\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [12]:
adata_sub.obs['azimuth_label'] = adata_sub.obs['cell_type']
adata_sub.obsm['X_umap_proj'] = adata_sub.obsm['X_umap']

In [13]:
adata_sub.write_h5ad("BLADDER_TEST")